In [17]:
# Now the Deep Learning initialization

import keras.backend as K
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Activation, Dropout, Input
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.core import Lambda
from keras.engine.topology import Layer, InputSpec

import os
from os import listdir, mkdir
from scipy.misc import imread
import numpy as np
from sklearn.utils import shuffle
from scipy.io import loadmat
import h5py
import sys

In [19]:
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=0.0005, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):
        if K.image_dim_ordering == "th":
            _, f, r, c = self.shape
        else:
            _, r, c, f = self.shape
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1),
            padding="same", pool_mode="avg")
        if K.image_dim_ordering == "th":
            summed = K.sum(pooled, axis=1, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=1)
        else:
            summed = K.sum(pooled, axis=3, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom

    def get_output_shape_for(self, input_shape):
        return input_shape
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [20]:
img_height = 54
img_width = 32
num_channels = 3
num_classes = 2
num_images_train = 207574
num_images_test = 216649

root = '../PKLot/PKLotSegmented/PUC/'

trainX = np.zeros((num_images_train, img_height, img_width, num_channels))
validX  = np.zeros((num_images_test, img_height, img_width, num_channels))

trainY = np.zeros(num_images_train)
validY = np.zeros(num_images_test)

counter_train = 0
counter_test = 0

if not os.path.isdir('weights'):
    mkdir('weights')

if not os.path.isdir(root):
    print('PKLot dataset folder missing... Exiting!!')
    sys.exit(0)

weathers = [w for w in listdir(root)]

#------------------------------------------------------------- Preparing Data
print('Preparing data...')
for weather in weathers:
    root_ = root + weather + '/'
    print("Inside folder: %s" % root_)
    days = [day for day in listdir(root_)]
    mid = int(len(days)/2)
    train_days = days[:mid]
    test_days = days[mid:]
    
    # add to train data
    for day in train_days:
        _root_ = root_ + day + '/'
        print("Inside folder: %s" % _root_)
        labels = [label for label in listdir(_root_)]
        for i, label in enumerate(labels):
            img_root = _root_ + label + '/'
            img_names = [img for img in listdir(img_root)]
            for j,img_name in enumerate(img_names):
                if (j+1) % 1000 == 0:
                    print("Inside folder: %s, image # %d" % (img_root, j+1))
                img = imread(img_root + img_name)
                img = np.resize(img, (img_height, img_width, num_channels))
                trainX[counter_train] = img
                trainY[counter_train] = 1-i
                counter_train += 1

    # add to test data
    for day in test_days:
        _root_ = root_ + day + '/'
        print("Inside folder: %s" % _root_)
        labels = [label for label in listdir(_root_)]
        for i, label in enumerate(labels):
            img_root = _root_ + label + '/'
            img_names = [img for img in listdir(img_root)]
            for j,img_name in enumerate(img_names):
                if (j+1) % 1000 == 0:
                    print("Inside folder: %s, image # %d" % (img_root, j+1))
                img = imread(img_root + img_name)
                img = np.resize(img, (img_height, img_width, num_channels))
                validX[counter_test] = img
                validY[counter_test] = 1-i
                counter_test += 1
    print('Done Loading dataset')

Preparing data...
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 6000
Inside fol

/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-21/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-16/
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-16/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-09-16/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-26/
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012

Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Empty/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-20/Occupied/, image # 8000
Insi

Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-19/Occupied/, image # 8000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-13/
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-13/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-13/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-13/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-13/Empty/, image # 4000
Inside folder: ../PKLot/PKL

Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 8000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 9000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 10000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 11000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 12000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 13000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-13/Empty/, image # 14000
Inside fold

IndexError: index 216649 is out of bounds for axis 0 with size 216649

In [29]:
trainX, trainY = shuffle(trainX, trainY)
validX, validY = shuffle(validX, validY)

# print('Saving to h5py')
# data = h5py.File('data.h5', 'w')
# data.create_dataset('trainX', data=trainX)
# data.create_dataset('trainY', data=trainY)
# data.create_dataset('validX', data=validX)
# data.create_dataset('validY', data=validY)
# data.close()
print('Done.')

Done.


In [30]:
def VGG_16():
    '''Model definition'''

    model = Sequential()
    model.add(Conv2D(64, (11, 11,), padding='valid', strides=(4,4), input_shape=(img_height,img_width,num_channels), name='conv1'))
    model.add(Activation('relu', name='relu1'))
    model.add(LocalResponseNormalization(name='norm1'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool1'))

    model.add(Conv2D(256, (5,5), padding='same', name='conv2'))
    model.add(Activation('relu', name='relu2'))
    model.add(LocalResponseNormalization(name='norm2'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name='conv3'))
    model.add(Activation('relu', name='relu3'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv4'))
    model.add(Activation('relu', name='relu4'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv5'))
    model.add(Activation('relu', name='relu5'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool5'))

    return model

In [31]:
def copy_mat_to_keras(kmodel, weights_path):
    '''Utility function
    Loads weights from .mat file to keras model
    '''

    kerasnames = [lr.name for lr in kmodel.layers]
    data = loadmat(weights_path, matlab_compatible=False, struct_as_record=False)
    layers = data['layers']
    description = data['meta'][0,0].classes[0,0].description

    prmt = (0,1,2,3)

    for i in range(layers.shape[1]):
        matname = layers[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            if len(layers[0,i][0,0].weights) > 0:
                l_weights = layers[0,i][0,0].weights[0,0]
                l_bias = layers[0,i][0,0].weights[0,1]
                f_l_weights = l_weights.transpose(prmt)
                assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
                assert (l_bias.shape[1] == 1)
                assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
                assert (len(kmodel.layers[kindex].get_weights()) == 2)
                kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

In [63]:
model = VGG_16()
# Load pre-trained weights from .mat file
#copy_mat_to_keras(model, 'weights/imagenet-vgg-f.mat')
copy_mat_to_keras(model,'imagenet-vgg-f.mat')
# Freeze the convolutional layers
for layer in model.layers:
    layer.trainable = False

model.add(Flatten())
model.add(Dropout(0.2, name='dropout6'))
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(0.2, name='dropout7'))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(0.2, name='dropout8'))
model.add(Dense(1, activation='sigmoid', name='predictions'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 11, 6, 64)         23296     
_________________________________________________________________
relu1 (Activation)           (None, 11, 6, 64)         0         
_________________________________________________________________
norm1 (LocalResponseNormaliz (None, 11, 6, 64)         0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 6, 3, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 6, 3, 256)         409856    
_________________________________________________________________
relu2 (Activation)           (None, 6, 3, 256)         0         
_________________________________________________________________
norm2 (LocalResponseNormaliz (None, 6, 3, 256)         0         
__________

In [64]:
batch_size = 128
epochs = 10
learning_rate = 0.001
weight_decay = 0.00005
nesterov = True
momentum = 0.99

In [65]:
#------------------------------------------------------------- Training

# optimizer
#sgd = SGD(lr=learning_rate, decay=weight_decay, momentum=momentum, nesterov=nesterov)
sgd=Adam(lr=learning_rate,beta_1=0.9,beta_2=0.999, epsilon=None, decay=weight_decay, amsgrad=False)

# Callbacks
tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
checkpointer = ModelCheckpoint(filepath="./weights/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(validX, validY),
          callbacks=[checkpointer, tb])


probs = model.predict(validX, verbose=1)
model.save('model.h5')
print('Training complete.')

Train on 207574 samples, validate on 216649 samples
Epoch 1/10
207574/207574 [==============================] - 70s 336us/step - loss: 0.1893 - acc: 0.9325 - val_loss: 0.0544 - val_acc: 0.9837

Epoch 00001: saving model to ./weights/checkpoint-01-0.05.hdf5
Epoch 2/10
207574/207574 [==============================] - 69s 334us/step - loss: 0.1526 - acc: 0.9459 - val_loss: 0.0542 - val_acc: 0.9832

Epoch 00002: saving model to ./weights/checkpoint-02-0.05.hdf5
Epoch 3/10
207574/207574 [==============================] - 69s 334us/step - loss: 0.1473 - acc: 0.9484 - val_loss: 0.0536 - val_acc: 0.9838

Epoch 00003: saving model to ./weights/checkpoint-03-0.05.hdf5
Epoch 4/10
207574/207574 [==============================] - 69s 333us/step - loss: 0.1467 - acc: 0.9487 - val_loss: 0.0537 - val_acc: 0.9835

Epoch 00004: saving model to ./weights/checkpoint-04-0.05.hdf5
Epoch 5/10
207574/207574 [==============================] - 69s 334us/step - loss: 0.1450 - acc: 0.9495 - val_loss: 0.0538 - val

In [ ]:
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report

#AUC curve
False_Positive_Rate, True_Positive_Rate, threshold = roc_curve(validY, probs)
aucTree = roc_auc_score(validY, probs)
plt.title('AUC Full Dataset')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(False_Positive_Rate, True_Positive_Rate, color='blue',label='AUC' % aucTree)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()
print(probs, len(probs))


#Precision Recall

precision, recall,_ = precision_recall_curve(validY, probs)
    
plt.plot(recall, precision,  color="green",label='Precision-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve Full Dataset')
plt.legend(loc="upper right")
plt.show()

In [ ]:
label = np.where(probs>0.5,1,0)
tn, fp, fn, tp = confusion_matrix(validY, label).ravel()
print("True Positives", tp)
print("False Positives", fp)
print("True Negatives", tn)
print("False Negatives", fn)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print( 'Accuracy:', accuracy_score(label,validY)*100)
print( 'F1 score:', f1_score(label,validY)*100)
print ('Recall:', recall_score(label,validY)*100)
print ('Precision:', precision_score(label,validY)*100)

In [ ]:
#Saliency Maps
class_idx = 0
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)
plt.imshow(trainX[30])

In [ ]:
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations
layer_idx = utils.find_layer_idx(model, 'predictions')
grads = visualize_saliency(model, layer_idx, filter_indices=class_idx, seed_input=validX[30])
plt.imshow(grads, cmap='jet')

In [ ]:
root = '../PKLot/PKLotSegmented/PUC/'
_root_ = root+'Sunny/'
# img_root = _root_ + label + '/'
img_name = '2012-09-16/Empty/2012-09-16_18_13_23#098.jpg'
img = imread(_root_ + img_name)

img_name = '2012-09-16/Occupied/2012-09-16_17_43_22#071.jpg'
img2 = imread(_root_ + img_name)

_root2 = root+'Rainy/'
img_r = '2012-11-09/Occupied/2012-11-09_13_11_50#057.jpg'
img3 = imread(_root2 + img_r)